In [ ]:
import numpy as np
import pandas as pd
import cv2
import random
import matplotlib.image as mpimg
import tensorflow as tf
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def model(joints):
    conv1_W = tf.Variable(tf.truncated_normal(shape=(11, 11, 3, 96)))
    conv1_b = tf.Variable(tf.zeros(96))
    conv1   = tf.nn.conv2d(joints, conv1_W, strides=[4, 4, 4, 4], padding='VALID') + conv1_b
    conv1 = tf.nn.relu(conv1)
    conv1 = tf.nn.local_response_normalization(conv1)
    conv1 = tf.nn.max_pool(conv1, ksize=[3, 2, 2, 3], strides=[3, 2, 2, 3], padding='VALID')
    
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 96, 256)))
    conv2_b = tf.Variable(tf.zeros(256))
    conv2   = tf.nn.conv2d(joints, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    conv2 = tf.nn.relu(conv2)
    conv2 = tf.nn.local_response_normalization(conv2)
    conv2 = tf.nn.max_pool(conv2, ksize=[3, 2, 2, 3], strides=[3, 2, 2, 3], padding='VALID')
    
    conv3_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 256, 384)))
    conv3_b = tf.Variable(tf.zeros(384))
    conv3   = tf.nn.conv2d(joints, conv3_W, strides=[1, 1, 1, 1], padding='VALID') + conv3_b
    conv3 = tf.nn.relu(conv3)
    
    conv4_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 384, 384)))
    conv4_b = tf.Variable(tf.zeros(384))
    conv4   = tf.nn.conv2d(joints, conv4_W, strides=[1, 1, 1, 1], padding='VALID') + conv4_b
    conv4 = tf.nn.relu(conv4)
    
    conv5_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 384, 256)))
    conv5_b = tf.Variable(tf.zeros(256))
    conv5   = tf.nn.conv2d(joints, conv5_W, strides=[1, 1, 1, 1], padding='VALID') + conv5_b
    conv5 = tf.nn.max_pool(conv5, ksize=[3, 2, 2, 3], strides=[3, 2, 2, 3], padding='VALID')
    conv5 = tf.nn.relu(conv5)
    conv5 = tf.nn.max_pool(conv5, ksize=[3, 2, 2, 3], strides=[3, 2, 2, 3], padding='VALID')
    
    conv5_inputs = int(np.prod(conv5.shape()[1:]))
    fc6_W = tf.Variable(tf.truncated_normal(shape=(conv5_inputs, 4096)))
    fc6_b = tf.Variable(tf.zeros(4096))
    fc6 = tf.matmul(joints, fc6_W) + fc6_b
    fc6 = tf.nn.relu(fc6)
    fc6 = tf.nn.dropout(fc6, 0.6)
    
    fc6_inputs = int(fc6.shape()[1])
    fc7_W = tf.Variable(tf.truncated_normal(shape=(fc6_inputs, 4096)))
    fc7_b = tf.Variable(tf.zeros(4096))
    fc7 = tf.matmul(fc6, fc7_W) + fc6_b
    fc7 = tf.nn.relu(fc7)
    fc7 = tf.nn.dropout(fc7, 0.6)
    
    fc7_inputs = int(np.prod(fc7.shape()[1]))
    fc8_W = tf.Variable(tf.truncated_normal(shape=(fc8_inputs, 14)))
    fc8_b = tf.Variable(tf.zeros(14))
    fc8 = tf.matmul(joints, fc8_W) + fc8_b
    
    return fc8

In [ ]:
joints = 0
# joints preprocessing still left

In [ ]:
Epoch = 100
rate = 0.0005
logits = model(joints)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)